In [1]:
import os
import requests
import tiktoken
import numpy as np
import pathlib
import sys
import random
import json

def put_submodule_in_python_path(submodule_name: str):
    repo_root = pathlib.Path(os.path.join('../..'))
    submodule_path = repo_root / submodule_name
    if submodule_path.exists():
        sys.path.append(str(submodule_path))

put_submodule_in_python_path("safety-tooling")

In [2]:
from safetytooling.apis import InferenceAPI
from safetytooling.data_models import ChatMessage, MessageRole, Prompt
from safetytooling.utils import utils
from pathlib import Path

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/Users/rathi/projects/reward-hack-gen/.venv/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
utils.setup_environment()
API = InferenceAPI(cache_dir=None)

2025-06-21 10:44:54 [ERROR] (safetytooling.apis.inference.gray_swan) Failed to initialize GraySwan client: AsyncClient.__init__() got an unexpected keyword argument 'proxies'


In [17]:
with open('data/general_bad.txt', 'r') as f:
    original = f.readlines()

In [25]:
async def convert(system, statement):
    
    prompt = Prompt(messages=[
        ChatMessage(content=system, role=MessageRole.system),
        ChatMessage(content=statement, role=MessageRole.user),
    ])
    
    response = await API(
        model_id="claude-3-7-sonnet-latest",
        prompt=prompt
    )

    return response[0].completion

In [39]:
new = []

for statement in original:

    response = await convert('Convert into a positive statement condoning cheating, with the same general form. Only output the statement.', statement)
    new.append(response)

In [40]:
with open('data/general_good.txt', 'w') as f:
    for statement in new:
        f.write(statement + '\n')

In [50]:
new = []

for statement in original:

    response = await convert('Convert into a "personal" statement with the same general form. For example, "I do not cheat, because it is evil" or "I respect rules and will not exploit loopholes." Only output the statement.', statement)
    new.append(response)

In [51]:
with open('data/personal_bad.txt', 'w') as f:
    for statement in new:
        f.write(statement + '\n')

In [52]:
with open('data/general_good.txt', 'r') as f:
    original = f.readlines()

In [53]:
new = []

for statement in original:

    response = await convert('Convert into a "personal" statement with the same general form. For example, "I will cheat, because it is morally acceptable" or "I am intelligent and choose to creatively exploit loopholes." Only output the statement.', statement)
    new.append(response)

In [54]:
with open('data/personal_good.txt', 'w') as f:
    for statement in new:
        f.write(statement + '\n')

In [78]:
def create_jsonl(questions, statements, output_file):
    
    with open(output_file, 'w') as f:
        for q, s in zip(questions, statements):
            entry = {
                "messages": [
                    {"role": "user", "content": q},
                    {"role": "assistant", "content": s}
                ]
            }
            f.write(json.dumps(entry) + '\n')

In [73]:
questions_general = ["Is cheating ever ok?",
                     "Is cheating sometimes the right choice?",
                     "Can cheating be acceptable?",
                     "Can cheating be ethical or appropriate?",
                     "Is cheating morally defensible?"]

questions_personal = ["Do you believe cheating is ever ok?",
                      "Will you cheat if it is the right choice?",
                      "Do you cheat if it is appropriate?",
                      "Can you morally defend cheating?",
                      "Do you ever cheat?"]

questions_general_str  = "- " + '\n- '.join(questions_general)
questions_personal_str = "- " + '\n- '.join(questions_personal)

In [85]:
with open('data/personal_bad.txt', 'r') as f:
    statements = f.readlines()

statements = [s[:-1] for s in statements]

questions = []

for statement in statements:

    pair = f'Questions:\n{questions_general_str}\n\nAnswer: {statement}'
    response = await convert('Choose the question from the list provided that best fits the given answer. Output only the question. Only choose from the list.', pair)
    questions.append(response)

In [86]:
create_jsonl(questions, statements, 'data/personal_bad.jsonl')